**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
! pip install scikit-video
! pip install opencv-python

     |████████████████████████████████| 2.3MB 8.9MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

This act funtion uses an $\epsilon$-greedy strategy. With probability 1-$\epsilon$, it will choose the action that maximizes the reward $a_{t+1} = arg \max_{a \in A} r(s_t, a)$ and with probability $\epsilon$ the algorithm will choose randomly an action among all the available actions.

This strategy deals with the exploration-exploitation dilemma. This ensures that the agent explore new actions even if he doesn't think it is the best yet.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 100 # set small when debugging
epochs_test= 100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**board** is a matrix (2d-array) representing the board of the game. It has the following caracteristics :
  

*   The size of the board is (N, N) where N = grid_size + 4. We add 4 to account for a border of width 2 in each direction.
*   For each positions, the matrix board contains a 0 if there is no reward, -1 if there is a negative reward (malus) and 0.5 if there is a positive reward (bonus).
*  A binomial distribution, with probability the temperature parameter, is used to decide where the malus and bonus are.
* If there is a bonus and a malus in the same case, then the case contains a 0.

**position** is another matrix that indicates where the player is at a certain time $t$. The matrix contains 0 everywhere except a 1 to indicate where the rat is.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # restart the game for each epoch
        state = env.reset()
        
        win, lose = 0, 0
        game_over = False
        
        while not game_over:
            a = agent.learned_act(state)
            
            state, reward, game_over = env.act(a)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/11.0. Average score (-1.5)
Win/lose count 13.5/13.0. Average score (-0.5)
Win/lose count 12.0/11.0. Average score (0.0)
Win/lose count 7.5/8.0. Average score (-0.125)
Win/lose count 11.0/14.0. Average score (-0.7)
Win/lose count 7.0/18.0. Average score (-2.4166666666666665)
Win/lose count 13.0/14.0. Average score (-2.2142857142857144)
Win/lose count 12.0/14.0. Average score (-2.1875)
Win/lose count 10.5/18.0. Average score (-2.7777777777777777)
Win/lose count 11.5/17.0. Average score (-3.05)
Win/lose count 9.5/16.0. Average score (-3.3636363636363638)
Win/lose count 6.0/16.0. Average score (-3.9166666666666665)
Win/lose count 10.5/16.0. Average score (-4.038461538461538)
Win/lose count 14.0/17.0. Average score (-3.9642857142857144)
Win/lose count 10.0/5.0. Average score (-3.3666666666666667)
Win/lose count 9.5/14.0. Average score (-3.4375)
Win/lose count 9.5/14.0. Average score (-3.5)
Win/lose count 7.5/11.0. Average score (-3.5)
Win/lose count 8.5/4.0. Average score

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r(s,a)+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation*}






**First equality:**

\begin{equation*}
\begin{split}
Q^{\pi}(s,a) & =E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=E_{p^{\pi}}[r(s, a) + \sum_{t=1}^{\infty} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&=E_{p^{\pi}}[r(s, a) + \gamma\sum_{t=0}^{\infty} \gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]  \\
&=E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a,s_{1}=s',a_{1}=a']] \\
&=E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}
\end{equation*}
  
**Second equality**

We assume that $\pi^*$ exists. Let $\alpha(s)=arg \max_{a} Q^*(s,a)$

\begin{equation*}
\begin{split}
Q^{*}(s,a)&=E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')] \\
& \leq E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')] \\
& \leq E_{s'\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')]  \\
& := Q^{\alpha}(s,a)  \\
\end{split}
\end{equation*}

By optimality of $Q^*$ we thus have $Q^* = Q^{\alpha}$, and so we conclude that

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \alpha(.|s,a)}[r(s,a)+\gamma Q^{\alpha}(s',a')]
\end{equation*}

**Third equality**

We want $Q(s,a,\theta)$ and $E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^*(s',a',\theta)]$ to be close.

Then we want $\Vert E_{s' \sim \pi^*(.|s,a)}[ r(s,a)+\gamma\max_{a'}Q(s',a',\theta)]-Q(s,a,\theta)\Vert^{2}$ to be small. 

Therefore we can define our loss as:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r(s,a)+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.memory = list()
        self.max_memory = max_memory

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            self.memory = self.memory[1:]
        self.memory.append(m)

    def random_access(self):
        if len(self.memory) == 0:
            return None
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    
    # add decaying epsilon method for question 10
    def decay_epsilon(self, e):
        self.epsilon *= e # e is the dacay percentage

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # sample minibatch of transistions
            # (state, next_state, action, reward, game_over)
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))
            
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, 0)))
                
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model

        model = Sequential()
        model.add(Dense(64, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4))
        
        model.compile('adam', "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/100 | Loss 0.0043 | Win/lose count 3.5/4.0 (-0.5)
Epoch 001/100 | Loss 0.0025 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/100 | Loss 0.0012 | Win/lose count 2.0/7.0 (-5.0)
Epoch 003/100 | Loss 0.0009 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/100 | Loss 0.0012 | Win/lose count 7.0/2.0 (5.0)
Epoch 005/100 | Loss 0.0055 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/100 | Loss 0.0015 | Win/lose count 5.5/2.0 (3.5)
Epoch 007/100 | Loss 0.0542 | Win/lose count 8.5/5.0 (3.5)
Epoch 008/100 | Loss 0.0041 | Win/lose count 6.5/2.0 (4.5)
Epoch 009/100 | Loss 0.0615 | Win/lose count 5.5/4.0 (1.5)
Epoch 010/100 | Loss 0.0033 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/100 | Loss 0.0014 | Win/lose count 6.0/5.0 (1.0)
Epoch 012/100 | Loss 0.0036 | Win/lose count 9.5/3.0 (6.5)
Epoch 013/100 | Loss 0.0009 | Win/lose count 15.5/3.0 (12.5)
Epoch 014/100 | Loss 0.0032 | Win/lose count 6.5/2.0 (4.5)
Epoch 015/100 | Loss 0.0019 | Win/lose count 11.0/3.0 (8.0)
Epoch 016/100 | Loss 0.0113 | Win/lose

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(128, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(128, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0046 | Win/lose count 1.0/2.0 (-1.0)
Epoch 001/100 | Loss 0.0031 | Win/lose count 2.5/8.0 (-5.5)
Epoch 002/100 | Loss 0.0009 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/100 | Loss 0.0034 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/100 | Loss 0.0007 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/100 | Loss 0.0009 | Win/lose count 1.5/2.0 (-0.5)
Epoch 006/100 | Loss 0.0011 | Win/lose count 5.0/2.0 (3.0)
Epoch 007/100 | Loss 0.0055 | Win/lose count 4.0/1.0 (3.0)
Epoch 008/100 | Loss 0.0054 | Win/lose count 5.0/6.0 (-1.0)
Epoch 009/100 | Loss 0.0041 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/100 | Loss 0.0026 | Win/lose count 5.0/4.0 (1.0)
Epoch 011/100 | Loss 0.0047 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/100 | Loss 0.0030 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/100 | Loss 0.0037 | Win/lose count 6.5/4.0 (2.5)
Epoch 014/100 | Loss 0.0356 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/100 | Loss 0.0028 | Win/lose count 3.0/1.0 (2.0)
Epoch 016/100 | Loss 0.0029 | Win/lose count 9.5/2.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 5.5/0. Average score (3.5)
Win/lose count 4.0/0. Average score (3.6666666666666665)
Win/lose count 8.5/0. Average score (4.875)
Win/lose count 10.0/0. Average score (5.9)
Win/lose count 3.5/0. Average score (5.5)
Win/lose count 8.5/0. Average score (5.928571428571429)
Win/lose count 0.5/0. Average score (5.25)
Win/lose count 0/0. Average score (4.666666666666667)
Win/lose count 2.0/0. Average score (4.4)
Win/lose count 2.5/0. Average score (4.2272727272727275)
Win/lose count 7.0/0. Average score (4.458333333333333)
Win/lose count 4.5/0. Average score (4.461538461538462)
Win/lose count 1.5/0. Average score (4.25)
Win/lose count 5.5/0. Average score (4.333333333333333)
Win/lose count 4.5/0. Average score (4.34375)
Win/lose count 2.0/0. Average score (4.205882352941177)
Win/lose count 4.0/0. Average score (4.194444444444445)
Win/lose count 5.5/0. Average score (4.2631578947368425)
Win/lose count 3.5/0. Average score 

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

The CNN DQN outperforms the DQN with fully conntected layers. A big problem is that the agent often gets stuck in parts of the board and doesn't explore anymore.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.decay_epsilon(decay)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))


                   # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error... 2 times same line ?!!
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        self.board[self.x, self.y] = 0 # erase reward from board
        game_over = self.t > self.max_time # game over if more than 500 steps

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over


    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # add 0.5 of reward in some cases of the board
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        # add -1 reward in some cases of the board
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 # if both malus and bonus on the same case -> put 0

        self.board = bonus + malus # board is 0 if no reward, -0.5 if malus, 1 if positive reward
        
        # lines added (put 0 poison/cheese outside the border)
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0181 | Win/lose count 13.5/44.20000000000006 (-30.70000000000006)
Epoch 001/100 | Loss 0.0166 | Win/lose count 9.0/53.000000000000036 (-44.000000000000036)
Epoch 002/100 | Loss 0.0242 | Win/lose count 19.0/23.90000000000001 (-4.900000000000009)
Epoch 003/100 | Loss 0.0095 | Win/lose count 21.0/20.900000000000006 (0.09999999999999432)
Epoch 004/100 | Loss 0.0127 | Win/lose count 22.0/14.099999999999985 (7.900000000000015)
Epoch 005/100 | Loss 0.0821 | Win/lose count 17.0/25.00000000000002 (-8.000000000000021)
Epoch 006/100 | Loss 0.0391 | Win/lose count 27.0/16.499999999999986 (10.500000000000014)
Epoch 007/100 | Loss 0.0156 | Win/lose count 22.0/15.299999999999981 (6.700000000000019)
Epoch 008/100 | Loss 0.0122 | Win/lose count 24.0/11.99999999999999 (12.00000000000001)
Epoch 009/100 | Loss 0.0080 | Win/lose count 22.5/15.599999999999989 (6.900000000000011)
Epoch 010/100 | Loss 0.0111 | Win/lose count 28.0/12.499999999999984 (15.500000000000016)
Epoch 011/100 | L

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 16.5/3.0. Average score (13.5)
Win/lose count 21.5/3.0. Average score (16.0)
Win/lose count 20.5/3.0. Average score (16.5)
Win/lose count 25.0/2.0. Average score (18.125)
Win/lose count 26.5/2.0. Average score (19.4)
Win/lose count 30.0/3.0. Average score (20.666666666666668)
Win/lose count 25.5/5.0. Average score (20.642857142857142)
Win/lose count 16.5/4.0. Average score (19.625)
Win/lose count 26.0/4.0. Average score (19.88888888888889)
Win/lose count 21.0/3.0. Average score (19.7)
Win/lose count 25.0/2.0. Average score (20.0)
Win/lose count 19.5/9.0. Average score (19.208333333333332)
Win/lose count 23.5/4.0. Average score (19.23076923076923)
Win/lose count 26.5/2.0. Average score (19.607142857142858)
Win/lose count 23.5/3.0. Average score (19.666666666666668)
Win/lose count 21.0/3.0. Average score (19.5625)
Win/lose count 28.0/3.0. Average score (19.88235294117647)
Win/lose count 17.0/6.0. Average score (19.38888888888889)
Win/lose count 19.0/5.0. Average score (19.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [22]:
import os
arr = os.listdir()
print(arr)

['.config', 'random29.mp4', 'random4.mp4', 'cnn_test_explore11.mp4', 'cnn_test35.mp4', 'random99.mp4', 'fc_test26.mp4', 'cnn_test31.mp4', 'random36.mp4', 'cnn_trainmodel.json', 'cnn_test_explore15.mp4', 'fc_train90.mp4', 'fc_trainmodel.json', 'cnn_test_explore32.mp4', 'cnn_test17.mp4', 'cnn_test54.mp4', 'cnn_test55.mp4', 'cnn_train0.mp4', 'random89.mp4', 'cnn_train_explore50.mp4', 'fc_test66.mp4', 'cnn_test_explore18.mp4', 'cnn_train10.mp4', 'fc_test97.mp4', 'fc_test18.mp4', 'random82.mp4', 'cnn_test90.mp4', 'cnn_test27.mp4', 'random55.mp4', 'random84.mp4', 'random5.mp4', 'cnn_test_explore21.mp4', 'cnn_test53.mp4', 'fc_test69.mp4', 'cnn_test_explore43.mp4', 'fc_test71.mp4', 'cnn_test33.mp4', 'fc_test14.mp4', 'random43.mp4', 'random39.mp4', 'fc_test48.mp4', 'cnn_test_explore61.mp4', 'cnn_test_explore47.mp4', 'fc_test82.mp4', 'fc_test67.mp4', 'fc_train50.mp4', 'fc_test12.mp4', 'random57.mp4', 'fc_test58.mp4', 'fc_test2.mp4', 'fc_train0.mp4', 'random38.mp4', 'cnn_test12.mp4', 'cnn_test_ex

***